# 세팅

In [ ]:
import os
import findspark
import pyspark


os.environ['SPARK_HOME'] = "/Users/yoosj/Desktop/datapipe/venv1/Analisis/spark-2.4.8-bin-hadoop2.7"
findspark.init()
sc = pyspark.SparkContext(appName="SparkExample")

## HDFS에 데이터 로드 후 Spark DF에 저장

시나리오: airbnb.csv파일을 로드하고 처리

In [ ]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [ ]:
hdfs_path = "hdfs://localhost;9000/user/hadoop"
file_path = hdfs_path + "airbnb.csv"
df = sqlContext.read.format("csv")\
    .option("header", 'ture')\
    .option("inferSchema", 'false')\
    .load(file_path)

해당 열의 null값 전처리

In [ ]:
df = df.na.fill(value="{채우고 싶은 값}", subset={"해당 열"})

일반적으로 readcsv 시, 전부 type=string으로 불러옴

그래서 이것들을 원본의 type에 맞게 전처리해 해줘야 함

In [ ]:
df_preprocessing = df.select(
    df.{열 이름}.cast({타입}()),
    df.{열 이름}.cast({타입}()),
    df.{열 이름}.cast({타입}())
)

# Assignment 1

In [ ]:
df1 = df_preprocessing.withColumn("total_score", df_preprocessing["overall_satisfaction"] + df_preprocessing["reviews"]*0.378)
df1 = df1.select("room_id", "host_idf", "total_score")
df1.sort("total_score", ascending=True).write.csv({저장 경로.csv})

# Assignment 2

In [ ]:
import pyspark.sql.functions as F

df2 = df_preprocessing
df2 = df2.groupBy("neighborhood")\
    .agg(
        F.mean("reviews").alias("avg of reviews"),\
        F.mean("overall_satisfaction").alias("avg of overall_satisfaction"),\
        F.mean("price").alias("avg of price"),\
        F.max("reviews").alias("max of reviews"),\
        F.min("reviews").alias("min of reviews"),\
        F.max("price").alias("max of price"),\
        F.min("price").alias("min of price"),\
    ).sort("neighborhood")

# Assignment 3

Bucketing 후 Col 이름 변경

In [ ]:

from pyspark.ml.feature import Bucketizer
from pyspark.sql.funcions import udf

df3 = df_preprocessing

bucketizer = Bucketizer(splits=[0,100,200,300,400,500,1000,5000], inputCol="price", outputCol="buckets")
df3_buck = bucketizer.setHandleInvalid("keep").transform(df3)

scope = {0.0: "0-100", 1.0: "100-200", 2.0: "200-300", 3.0: "300-400", 4.0: "4000-500", 5.0: "500-1000", 6.0: "1000-5000"}
udf_scope = udf(lambda x: scope[x], StringType())
df3_buck = df3_buck.witColumn("price_range", udf_scope("buckets"))

df3_compl = df3_buck.selelct("room_idf", "price", "buckets", "price_range")

Grouping 후 집계

In [ ]:
df3_compl = df3_compl.groupBy("buckets", "price_range")\
    .agg()

In [ ]:
df3_length = df3_compl.select('*', F.size("neighborhood").alias('length'))